<a href="https://colab.research.google.com/github/TannaPrasanthkumar/CodeGenerator-Using-RAG/blob/main/CodeGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_core langchain_chroma langchain_google_genai langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107

In [ ]:
import os
import langchain

In [ ]:
from google.colab import userdata

In [ ]:
GITHUB_REPO  = userdata.get("GITHUB_REPO")

GITHUB_ACCESS_TOKEN = userdata.get("GITHUB_ACCESS_TOKEN")

In [ ]:
import requests
import time

def crawl_github_repo(repo_url, github_token):

    ignore_list = ['__init__.py']

    api_url = f"https://api.github.com/repos/{repo_url}/contents"
    headers = {
        "Accept": "application/vnd.github.v3+json",
        "Authorization": f"Bearer {github_token}"
    }

    def get_files(url):
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        files = []
        for item in response.json():
            if item['type'] == 'file' and (item['name'] not in ignore_list) and (item['name'].endswith('.py') or item['name'].endswith('.ipynb')):
                files.append(item['html_url'])
            elif item['type'] == 'dir' :
                files.extend(get_files(item['url']))
                time.sleep(0.1)  # Avoid hitting rate limits
        return files

    return get_files(api_url)

code_files_urls = crawl_github_repo(GITHUB_REPO, GITHUB_ACCESS_TOKEN)


In [ ]:
print(code_files_urls)

['https://github.com/GoogleCloudPlatform/generative-ai/blob/main/.github/workflows/issue_assigner/assign_issue.py', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/conversation/data-store-status-checker/data_store_checker.ipynb', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/embedding-similarity-visualization.ipynb', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_embeddings_tuning.ipynb', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_multimodal_embeddings.ipynb', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/use-cases/outlier-detection/bq-vector-search-log-outlier-detection.ipynb', 'https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-quickstart.ipynb', 'https://github.com/GoogleCloudPlatform/generative

In [ ]:
for url in code_files_urls:
  print(url)

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/.github/workflows/issue_assigner/assign_issue.py
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/conversation/data-store-status-checker/data_store_checker.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/embedding-similarity-visualization.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro-textemb-vectorsearch.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_embeddings_tuning.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/intro_multimodal_embeddings.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/use-cases/outlier-detection/bq-vector-search-log-outlier-detection.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/embeddings/vector-search-quickstart.ipynb
https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/chat-

In [ ]:
print(len(code_files_urls))

220


In [ ]:
# Save                                                                                               URLs to a file
with open('code_files_urls.txt', 'w') as f:
    for url in code_files_urls:
        f.write(f"{url}\n")

In [ ]:
import requests
import nbformat
import json

In [ ]:
import requests
import nbformat
from langchain.schema import Document

def extract_python_code_from_ipynb(github_url, cell_type="code"):

    raw_url = github_url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')

    response = requests.get(raw_url)
    response.raise_for_status()

    notebook_content = response.text
    notebook = nbformat.reads(notebook_content, as_version = nbformat.NO_CONVERT)

    python_code = ""  # Initialize to an empty string

    for cell in notebook.cells:
        if cell.cell_type == cell_type:
            python_code += cell.source + "\n"  # Concatenate code with new line

    return python_code


code_strings = []

for i in range(len(code_files_urls)):

    if code_files_urls[i].endswith(".ipynb"):

        content = extract_python_code_from_ipynb(code_files_urls[i], "code")
        doc = Document(page_content=content, metadata={"url" : code_files_urls[i], "file_index" : i})
        code_strings.append(doc)

# Access the first code string
print(code_strings[0])

/usr/local/lib/python3.10/dist-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


page_content='%pip install --upgrade google-cloud-discoveryengine humanize
import sys

if "google.colab" in sys.modules:
    from google.auth import default
    from google.colab import auth

    auth.authenticate_user()
    creds, _ = default()
else:
    # Otherwise, attempt to discover local credentials as described on https://cloud.google.com/docs/authentication/application-default-credentials
    pass
import humanize
import time
import re
from typing import List, Optional

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1beta as discoveryengine


def _call_list_documents(
    project_id: str, location: str, datastore_id: str, page_token: Optional[str] = None
) -> discoveryengine.ListDocumentsResponse:
    """Build the List Docs Request payload."""
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengin

In [ ]:
print(len(code_files_urls))

220


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 2000,
    chunk_overlap = 200
)

In [ ]:
chunks = text_splitter.split_documents(code_strings)

In [ ]:
key = userdata.get("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model = "models/embedding-001",
    google_api_key = key
)

In [ ]:
!pip install langchain_chroma

In [ ]:
from langchain_chroma import Chroma

In [ ]:
import os
from datetime import datetime

# Create a unique directory name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
persistent_directory = f"/content/chroma_db_{timestamp}"

# Ensure the directory exists
os.makedirs(persistent_directory, exist_ok=True)

# Initialize Chroma
db = Chroma(
    collection_name="code_files",
    embedding_function=embeddings,
    persist_directory=persistent_directory
)

print(f"Chroma DB initialized at: {persistent_directory}")

Chroma DB initialized at: /content/chroma_db_20240816_060615


In [ ]:
db.add_documents(chunks)

['fa2a2e71-70dd-4a75-95f1-87ee69c5c99b',
 'c9d2a00d-eaf8-40e7-bc94-d6c608cd5b50',
 'd01309a0-6b1e-48bb-9fae-78597cbe54a5',
 '73156ae8-29ae-4de0-a4dd-8377767e3421',
 'a0ffc1a2-e813-4f7c-b33d-e27e3083e15f',
 '81886bb1-b8e7-49f2-99ae-9e03aefccaf2',
 'bbfec423-02ac-4c81-a191-233c56b84402',
 '2ae62090-7633-421f-908f-f2693761c682',
 '4c9cada4-d7c1-4a44-bf80-b6764429ca56',
 '9648dade-4dd1-4b32-9e55-2f14e0b301c7',
 'c41f54e7-dcee-4dcd-ab26-4ae7ec4853f9',
 '8d15c150-22f6-4e49-a946-1a4ce711013d',
 '76168c0c-ea9a-42cd-933b-91d244547574',
 'aa9bc262-7fac-4d55-98f5-3e318a86c0bf',
 '97dbd7f2-249d-4870-8da5-8a8375d4f043',
 '85d955b8-04c2-4d79-867a-9991aa9f1801',
 'fd601af3-2f42-4889-aa3f-35e9db8339b7',
 'c03c090f-806c-43a8-b9e0-f6fa5927e041',
 'd451b584-d950-4a05-aeea-b15b88f28746',
 'cb793295-43c2-42e0-8535-b323f65eb978',
 '7e691688-bd5b-48e6-9bf7-7e3ad1ee062c',
 '95001cc9-9cba-4757-a2a9-4b002de42791',
 '086db96a-0501-448e-9131-4396502e374b',
 '34a721ea-cc2f-4f78-bf3e-a29aa75cffe3',
 '67ae925b-a078-

In [ ]:
retriever = db.as_retriever(search_type = "similarity", kwargs = {"K":2})

In [ ]:
question = "write a python fucntion that will take two numbers as parameters and return the sum"

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
# Zero Shot prompt template
prompt_zero_shot = """
    You are a proficient python developer. Respond with the syntactically correct & concise code for to the question below.

    Question:
    {question}

    Output Code :
    """

prompt_prompt_zero_shot = PromptTemplate(
input_variables = ["question"],
template = prompt_zero_shot,
)

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
prompt_RAG = """
    You are a proficient python developer. Respond with the syntactically correct code for to the question below. Make sure you follow these rules:
    1. Use context to understand the APIs and how to use it & apply.
    2. Do not add license information to the output code.
    3. Do not include colab code in the output.
    4. Ensure all the requirements in the question are met.

    Question:
    {question}

    Context:
    {context}

    Helpful Response :
    """

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
from langchain_google_genai import GoogleGenerativeAI

model = GoogleGenerativeAI(
    model = "models/gemini-1.5-flash",
    temperature = 0.2,
    google_api_key = key
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_prompt_zero_shot | model | output_parser

In [ ]:
question = "write a python fucntion that will take two numbers as parameters and return the sum"

In [ ]:
response = chain.invoke(question)

In [ ]:
print(response)

```python
def sum_two_numbers(num1, num2):
  """Returns the sum of two numbers."""
  return num1 + num2
```
